# Diagramas de Sankey
---

En esta celda, vamos a utilizar los datos de esta primera iteración del modelado para analizar como se dividió todo el flujo de estudiantes del curso entre la entrega de tareas de evaluación continua del curso y respecto a las notas que obtuvieron

## Configuración

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

DATA_PATH = "/home/carlos/Documentos/TFG/spark-workspace/data/datasets"


## Carga de datos


In [2]:
df_entregas = pd.read_parquet(f"{DATA_PATH}/dataset_1.1.parquet")
df_notas = pd.read_parquet(f"{DATA_PATH}/dataset_1.2.parquet")

# Borrar columnas innecesarias
#=========================================================================

df_entregas = df_entregas.drop(columns=["num_entregas"])
df_notas = df_notas.drop(columns=["nota_media"])

# Reorganizar columnas
#=========================================================================

# Move "Test Complejidad" in df_entregas to the n-2 position
columns_entregas = df_entregas.columns.tolist()
columns_entregas.remove("Test Complejidad")
columns_entregas.insert(-2, "Test Complejidad")
df_entregas = df_entregas[columns_entregas]

# Move "Test Complejidad (nota)" in df_notas to the n-2 position
columns_notas = df_notas.columns.tolist()
columns_notas.remove("Test Complejidad (nota)")
columns_notas.insert(-2, "Test Complejidad (nota)")
df_notas = df_notas[columns_notas]

# Move "abandona" in df_entregas to the last position
columns_entregas.remove("abandona")
columns_entregas.append("abandona")
df_entregas = df_entregas[columns_entregas]

# Move "abandona" in df_notas to the last position
columns_notas.remove("abandona")
columns_notas.append("abandona")
df_notas = df_notas[columns_notas]

# Mostrar datos
#=========================================================================

display (df_entregas.head())
display(df_notas.head())


,userid,Test Expr.,Act. 02 - Elecciones,Act. 03 - Catalan,Act. 04 - Primos,Act. 05 - Vectores,Test Complejidad,Act. 07,abandona
0,e1f1d0f48ca77093f9d66cefd325504245277db3e6c145...,1,1,1,1,0,1,1,0
1,b5de2bb5b8538b199d6b3f0ecb32daa8a9d730ccc484db...,1,1,1,1,1,1,1,0
2,90a634296aff946e9d045997d512d2b77dbc01880715c1...,1,1,1,1,1,1,1,1
3,b6b2a12e84ea8203775195ed2bb4e99c5788053782b0bd...,1,1,1,1,1,1,1,0
4,fd96e32a94a932f45eb32933d9ffeb71f4addf9153a76b...,1,1,1,0,1,1,1,0


,userid,Test Expr. (nota),Act. 02 - Elecciones (nota),Act. 03 - Catalan (nota),Act. 04 - Primos (nota),Test Complejidad (nota),Act. 05 - Vectores (nota),Act. 07 (nota),abandona
0,e1f1d0f48ca77093f9d66cefd325504245277db3e6c145...,10.0,10.0,7.0,10.0,5.00000,NaN,8.50,0
1,b5de2bb5b8538b199d6b3f0ecb32daa8a9d730ccc484db...,10.0,10.0,10.0,10.0,6.00000,10.0,6.25,0
2,90a634296aff946e9d045997d512d2b77dbc01880715c1...,10.0,10.0,10.0,9.0,8.66667,5.0,6.00,1
3,b6b2a12e84ea8203775195ed2bb4e99c5788053782b0bd...,10.0,10.0,10.0,10.0,7.33333,10.0,10.00,0
4,fd96e32a94a932f45eb32933d9ffeb71f4addf9153a76b...,6.0,10.0,10.0,NaN,6.00000,10.0,0.00,0


## Diagramas


In [ ]:

PASS_MARK = 5  

# ─────────────── FUNCIONES AUXILIARES ───────────────
def _build_transition_counts(df, status_mapper):
    """
    Devuelve:
        nodes  · lista única de (<actividad>, <estado>)
        links  · listas paralelas: source, target, value
    `status_mapper` recibe un valor de la celda y devuelve la etiqueta de estado.
    """
    # Mantener solo las columnas de actividad (ignora userid / abandona…)
    act_cols = [c for c in df.columns if c not in {"userid", "abandona"}]

    # Construir el set de nodos y un dict (<act>, <estado>) → idx
    nodes = [(act, st) 
             for act in act_cols
             for st in ("Sí", "No")]          # placeholder, se sobreescribirá
    node_idx = {}

    # Lo rellenaremos dinámicamente para evitar nodos que nunca aparecen
    source, target, value = [], [], []

    # Recorremos pares consecutivos de actividades
    for c1, c2 in zip(act_cols[:-1], act_cols[1:]):
        trans = {}
        for _, row in df[[c1, c2]].iterrows():
            st1 = status_mapper(row[c1])
            st2 = status_mapper(row[c2])
            trans[(c1, st1, c2, st2)] = trans.get((c1, st1, c2, st2), 0) + 1

        # Registrar nodos y enlaces
        for (a1, s1, a2, s2), cnt in trans.items():
            # registrar nodos si no existen
            for a, s in ((a1, s1), (a2, s2)):
                if (a, s) not in node_idx:
                    node_idx[(a, s)] = len(node_idx)
            source.append(node_idx[(a1, s1)])
            target.append(node_idx[(a2, s2)])
            value.append(cnt)

    # Ordenar nodos en la lista final según su índice
    inv_idx = {v: k for k, v in node_idx.items()}
    nodes_sorted = [inv_idx[i] for i in range(len(inv_idx))]
    labels = [f"{act}<br>{st}" for act, st in nodes_sorted]

    return labels, source, target, value

def _plot_sankey(labels, source, target, value, title):
    link_colors = ["rgba(150,150,150,0.4)"] * len(source)
    fig = go.Figure(go.Sankey(
        arrangement="snap",
        node=dict(label=labels, pad=15, thickness=16,
                  color="rgba(44, 160, 101, 0.8)"),
        link=dict(source=source, target=target, value=value,
                  color=link_colors)))
    fig.update_layout(title_text=title, font_size=12, height=600)
    fig.show()

# ─────────────── DIAGRAMA 1 · ENTREGAS ───────────────
labels_ent, src_ent, tgt_ent, val_ent = _build_transition_counts(
    df_entregas,
    status_mapper=lambda v: "Entregó" if v == 1 else "No entregó"
)
_plot_sankey(labels_ent, src_ent, tgt_ent, val_ent,
             "Flujo de ENTREGAS por actividad")

# ─────────────── DIAGRAMA 2 · NOTAS ───────────────
labels_not, src_not, tgt_not, val_not = _build_transition_counts(
    df_notas,
    status_mapper=lambda v: "Aprobó" if (pd.notna(v) and v >= PASS_MARK) else "Suspendió"
)
_plot_sankey(labels_not, src_not, tgt_not, val_not,
             "Flujo de APROBADOS por actividad")
# ──────────────────────────────────────────────────────────────────────────────


##  Interpretación de los diagramas de Sankey


### 1. Flujo **ENTREGÓ / NO ENTREGÓ**

| Observación | Implicación pedagógica |
|-------------|------------------------|
| Predomina una **columna verde continua**: la inmensa mayoría (≈ 85-90 %) entrega todas las actividades de principio a fin. | El alumnado que empieza entregando tiende a mantenerse comprometido: buen síntoma de motivación sostenida. |
| El grupo que **no entrega desde la primera tarea (≈ 10-15 %) permanece casi estático**: los hilos grises apenas vuelven al verde. | La _no entrega inicial_ es un **indicador precoz de riesgo de inactividad/abandono**. Intervenciones muy tempranas (recordatorios, tutorías) son clave. |
| Transiciones inversas (de “Entregó” a “No entregó”) son testimoniales. | El abandono total tras haber comenzado es poco frecuente; conviene centrar los esfuerzos en quienes nunca arrancan. |

---

### 2. Flujo **APROBÓ / SUSPENDIÓ**

| Observación | Implicación pedagógica |
|-------------|------------------------|
| Las tres primeras actividades muestran **alto porcentaje de aprobados**. | El nivel inicial parece adecuado y motiva al alumnado. |
| En **“Test Complejidad”** y **“Act. 05 – Vectores”** aparecen **“cuellos de botella”**: aumentan bruscamente los hilos grises (suspensos), incluso de estudiantes que venían aprobando. | Son los **puntos críticos** del curso. Requieren refuerzo (material extra, ejemplos guiados, feedback inmediato). |
| Existen hilos que retornan de “Suspendió” a “Aprobó” en la actividad final, pero son mucho más finos que los que se mantienen en suspenso. | Hay cierta capacidad de recuperación, pero limitada: cuanto antes se intervenga, mayor probabilidad de éxito. |

---

### 3. Conclusiones operativas

1. **Detección temprana**  
   - Una “No entrega” o “Suspenso” inicial debería disparar alertas automáticas y tutorías personalizadas.

2. **Refuerzo focalizado**  
   - Diseñar estrategias específicas para _Test Complejidad_ y _Vectores_ (clases extra, vídeos paso a paso, rúbricas detalladas).

3. **Seguimiento longitudinal**  
   - El Sankey muestra herencia de estado: los primeros eventos condicionan los siguientes. Monitorizar ≈ en tiempo real permite cortar cadenas de suspensos/abandonos.

4. **Motivación del alumnado**  
   - Compartir métricas positivas (alta tasa de entrega, recuperación posible) puede fomentar la perseverancia y el auto-concepto de logro.

En conjunto, los diagramas confirman que **el comportamiento académico es altamente secuencial**: detectar desajustes pronto y actuar justo donde se producen los mayores descensos (complejidad y vectores) es la estrategia más prometedora para reducir el abandono y aumentar el rendimiento global.
